In [1]:
# 导入必要的库
import pandas as pd

In [8]:
#!pip3 install --user emoji


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# 读入数据（相对路径、绝对路径）
# 这里通过pd.read_csv()函数读取CSV文件，lineterminator参数用于指定换行符
df = pd.read_csv(r'covid19_tweets.csv', lineterminator='\n')
df_cleaned = pd.read_csv(r'covid19_tweets_cleaned_final.csv', lineterminator='\n')

In [3]:
import re, emoji
import  collections

all_emo = []
for t in df['text']:                     # df 为微博数据框
    all_emo.extend([m['emoji'] for m in emoji.emoji_list(str(t))])

counter = collections.Counter(all_emo)
emo_df  = (pd.DataFrame(counter.items(), columns=['emoji','freq'])
           .assign(name=lambda d: d['emoji'].apply(emoji.demojize))
           .sort_values('freq', ascending=False))
emo_df

,emoji,freq,name
55,😷,1207,:face_with_medical_mask:
49,😂,1113,:face_with_tears_of_joy:
18,👇,744,:backhand_index_pointing_down:
92,🤣,611,:rolling_on_the_floor_laughing:
14,🦠,527,:microbe:
...,...,...,...
853,🤲🏾,1,:palms_up_together_medium-dark_skin_tone:
1208,💴,1,:yen_banknote:
855,👐🏻,1,:open_hands_light_skin_tone:
858,🐹,1,:hamster:


In [4]:
raw_mask = df.text.str.count("😷|👍").sum()
clean_mask = df_cleaned.text.str.count("😷|👍").sum()
print(raw_mask, clean_mask)

1423 0


In [5]:
raw_pos = df.text.str.count("😊|👍").sum()
clean_pos = df_cleaned.text.str.count("😊|👍").sum()
print(raw_pos, clean_pos)

357 0


In [6]:
# 1. 查看数据集的基本信息
# 查看前几行数据
display(df.head())
# df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet\r\r
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False\r\r
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False\r\r
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False\r\r
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False\r\r
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False\r\r


In [7]:
# 查看数据集的基本信息（列名、数据类型、缺失值情况等）
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179108 entries, 0 to 179107
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         179108 non-null  object
 1   user_location     142337 non-null  object
 2   user_description  168822 non-null  object
 3   user_created      179108 non-null  object
 4   user_followers    179108 non-null  int64 
 5   user_friends      179108 non-null  int64 
 6   user_favourites   179108 non-null  int64 
 7   user_verified     179108 non-null  bool  
 8   date              179108 non-null  object
 9   text              179108 non-null  object
 10  hashtags          127774 non-null  object
 11  source            179031 non-null  object
      179108 non-null  object
dtypes: bool(1), int64(3), object(9)
memory usage: 16.6+ MB


In [8]:
#每列缺失行数占总行数的百分比
df.isnull().mean()*100

user_name            0.000000
user_location       20.530071
user_description     5.742904
user_created         0.000000
user_followers       0.000000
user_friends         0.000000
user_favourites      0.000000
user_verified        0.000000
date                 0.000000
text                 0.000000
hashtags            28.660920
source               0.042991
is_retweet\r\r       0.000000
dtype: float64

In [9]:
df['date'].head()

0    2020-07-25 12:27:21
1    2020-07-25 12:27:17
2    2020-07-25 12:27:14
3    2020-07-25 12:27:10
4    2020-07-25 12:27:08
Name: date, dtype: object

In [10]:
df['date'] = pd.to_datetime(df['date']) 
df['date'].head()

0   2020-07-25 12:27:21
1   2020-07-25 12:27:17
2   2020-07-25 12:27:14
3   2020-07-25 12:27:10
4   2020-07-25 12:27:08
Name: date, dtype: datetime64[ns]

In [5]:
# 查看数据的列名
print(df.columns)
# 查看数据的行数和列数
print(df.shape)

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet\r\r'],
      dtype='object')
(179108, 13)


In [6]:
# 2. 数据切割
# 提取特定的列，例如提取 'user_name' 和 'text' 列
df_selected = df[['user_name', 'text']]
display(df_selected.head())

,user_name,text
0,ᏉᎥ☻լꂅϮ,If I smelled the scent of hand sanitizers toda...
1,Tom Basile 🇺🇸,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,Time4fisticuffs,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,ethel mertz,@brookbanktv The one gift #COVID19 has give me...
4,DIPR-J&K,25 July : Media Bulletin on Novel #CoronaVirus...


In [7]:
# 按条件切割：例如，提取推文长度大于100的记录
df_long_tweets = df[df['text'].str.len() > 100]
display(df_long_tweets.head())

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet\r\r
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False\r\r
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False\r\r
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False\r\r
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False\r\r
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False\r\r


In [15]:
df_long_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162236 entries, 0 to 179107
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         162236 non-null  object
 1   user_location     129387 non-null  object
 2   user_description  152981 non-null  object
 3   user_created      162236 non-null  object
 4   user_followers    162236 non-null  int64 
 5   user_friends      162236 non-null  int64 
 6   user_favourites   162236 non-null  int64 
 7   user_verified     162236 non-null  bool  
 8   date              162236 non-null  object
 9   text              162236 non-null  object
 10  hashtags          110904 non-null  object
 11  source            162164 non-null  object
       162236 non-null  object
dtypes: bool(1), int64(3), object(9)
memory usage: 16.2+ MB


In [8]:
# 3. 数据合并
# 创建一个示例数据框，用于合并演示
new_data = pd.DataFrame({
    'user_name': ['Tom Basile 🇺🇸', 'Time4fisticuffs', 'ethel mertz'],
    'location': ['USA', 'UK', 'Canada']
})

In [9]:
# 按照 'user_name' 列将数据合并
df_merged = pd.merge(df_selected, new_data, on='user_name', how='left')
display(df_merged.head())

,user_name,text,location
0,ᏉᎥ☻լꂅϮ,If I smelled the scent of hand sanitizers toda...,NaN
1,Tom Basile 🇺🇸,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,USA
2,Time4fisticuffs,@diane3443 @wdunlap @realDonaldTrump Trump nev...,UK
3,ethel mertz,@brookbanktv The one gift #COVID19 has give me...,Canada
4,DIPR-J&K,25 July : Media Bulletin on Novel #CoronaVirus...,NaN


In [10]:
# 4. 保存数据
# 将df保存为Excel文件
#df.to_excel('tweets.xlsx', index=False)